<a href="https://colab.research.google.com/github/hanggao811/AnomalyDetectionLIGO/blob/main/AE_simplest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization, Input
from matplotlib import pyplot as plt
from sklearn import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import glob
from sklearn.base import BaseEstimator, RegressorMixin
from tensorflow.keras.models import load_model as keras_load_model
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# Set random seed
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)
# Mount to googlecolab
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/LIGO/DatasetSplitting/Datasplitting01/dataset')


Mounted at /content/drive


In [ ]:
# Load data and normalize it
background_train = np.load('background_train.npy')
stds_train = np.std(background_train, axis=-1)[:, :, np.newaxis]
background_train = background_train/stds_train
background_train = np.swapaxes(background_train, 1, 2)
background_test = np.load('background_test.npy')
stds_test = np.std(background_test, axis=-1)[:, :, np.newaxis]
background_test = background_test/stds_test
background_test = np.swapaxes(background_test, 1, 2)

bbh_train = np.load('bbh_train.npy')
stds_train = np.std(bbh_train, axis=-1)[:, :, np.newaxis]
bbh_train = bbh_train/stds_train
bbh_train = np.swapaxes(bbh_train, 1, 2)
bbh_test = np.load('bbh_test.npy')
stds_test = np.std(bbh_test, axis=-1)[:, :, np.newaxis]
bbh_test = bbh_test/stds_test
bbh_test = np.swapaxes(bbh_test, 1, 2)

sglf_train = np.load('sglf_train.npy')
stds_train = np.std(sglf_train, axis=-1)[:, :, np.newaxis]
sglf_train = sglf_train/stds_train
sglf_train = np.swapaxes(sglf_train, 1, 2)
sglf_test = np.load('sglf_test.npy')
stds_test = np.std(sglf_test, axis=-1)[:, :, np.newaxis]
sglf_test = sglf_test/stds_test
sglf_test = np.swapaxes(sglf_test, 1, 2)


In [ ]:
# Create train and test datasets
x_train = background_train
y_train = background_train
x_test = background_test
y_test = background_test

print(f'x train/test shapes: {x_train.shape} {x_test.shape}')
print(f'y train/test shapes: {y_train.shape} {y_test.shape}')


x train/test shapes: (80000, 200, 2) (20000, 200, 2)
y train/test shapes: (80000, 200, 2) (20000, 200, 2)


SOLUTION GIVEN

In [ ]:
tf.config.experimental.enable_op_determinism()

class Autoencoder(BaseEstimator, RegressorMixin):
    def __init__(self,filters=3, kernel_size=3, num_heads=4, key_dim=4, latent_dim=8, dropout=0.1,N_encoder=1, N_decoder=1,input_shape=(200,2),lr=1e-4,epochs=70,batch_size=700):
        super().__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.latent_dim = latent_dim
        self.dropout = dropout
        self.N_encoder = N_encoder
        self.N_decoder = N_decoder
        self.input_shape = input_shape
        self.lr = lr
        self.epochs = epochs
        self.batch_size = batch_size
        self.model_ = None

    # def positional_encoding(self, inputs):
    #     embed_size = inputs.shape[-1]
    #     input_length = inputs.shape[-2]
    #     assert embed_size % 2 == 0, 'embed_size must be even'
    #     p, i = np.meshgrid(np.arange(input_length), np.arange(embed_size // 2))
    #     pos_emb = np.empty((1,input_length, embed_size))
    #     pos_emb[0,:,::2] = np.sin(p/(10000**(2*i/embed_size))).T
    #     pos_emb[0,:,1::2] = np.cos(p/(10000**(2*i/embed_size))).T
    #     self.pos_encoding = tf.constant(pos_emb.astype(np.float32))
    #     return inputs + self.pos_encoding[:, :input_length, :]

    def build_model(self):
        inputs = keras.Input(shape=self.input_shape)
        # inputs = self.positional_encoding(inputs)

        # Encoder
        x = inputs
        for _ in range(self.N_encoder):
            x = layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, activation='relu', padding='same')(x)
            x = layers.MultiHeadAttention(num_heads=self.num_heads, key_dim=self.key_dim)(x, x)

        x = layers.Flatten()(x)
        x = layers.Dense(self.latent_dim, activation='relu')(x)

        for _ in range(self.N_decoder):
            num_units = np.prod(self.input_shape)
            x = layers.Dense(num_units,activation='tanh')(x)

        x = layers.Reshape(self.input_shape)(x)
        outputs = layers.Dense(self.input_shape[-1])(x)

        model = keras.Model(inputs,outputs)
        model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=self.lr))
        return model

    def predict(self, X):
        return self.model_.predict(X, batch_size=self.batch_size)

    def fit(self, X, y=None, **fit_params):
        self.model_ = self.build_model()
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
        if "callbacks" in fit_params:
            fit_params["callbacks"].append(early_stopping)
        else:
            fit_params["callbacks"] = [early_stopping]

        self.model_.fit(
            X, X,
            batch_size=self.batch_size,
            epochs=self.epochs,
            shuffle=False,
            validation_split=0.2,
            **fit_params
        )
        return self

    def score(self, X,y=None):
        recon = self.model_.predict(X)
        loss = np.mean((X - recon)**2)
        return -loss

In [ ]:
model = Autoencoder()

from scipy.stats import randint, uniform
param_grid = {
    "latent_dim": [8,16,32,64],          # latent_dim random integer between 8 and 64
    "filters": [1,4, 8, 16],              # number of Conv1D filters
    "num_heads": [2,4,6],             # number of attention heads
    "key_dim": [2,4,6],               # dimension of each head
    "dropout": [0.1],            # dropout random between 0.1 and 0.5
    "N_encoder": [1,2,3,4],             # number of transformer encoder blocks
    "N_decoder": [1,2,3,4],             # number of dense decoder blocks
    "lr": [1e-4],               # learning rate random between 1e-5 and 1e-4
    "epochs":[50],
    "batch_size":[700]
}

search = GridSearchCV(
    Autoencoder(input_shape=(200,2)),
    param_grid=param_grid,  # 👈 Notice: Now it’s param_grid, not param_distributions
    cv=2,
    verbose=2
)
search.fit(x_train,x_train)
print("Best parameters found:")
print(search.best_params_)

print("Best score:")
print(search.best_score_)


Streaming output truncated to the last 5000 lines.
Epoch 3/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 349ms/step - loss: 1.0005 - val_loss: 1.0005
Epoch 4/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 360ms/step - loss: 1.0005 - val_loss: 1.0005
Epoch 5/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 354ms/step - loss: 1.0005 - val_loss: 1.0005
Epoch 6/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 349ms/step - loss: 1.0005 - val_loss: 1.0005
Epoch 7/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 20s 349ms/step - loss: 1.0005 - val_loss: 1.0005
Epoch 8/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 12s 360ms/step - loss: 1.0005 - val_loss: 1.0005
Epoch 9/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 354ms/step - loss: 1.0005 - val_loss: 1.0005
Epoch 10/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 12s 361ms/step - loss: 1.0005 - val_loss: 1.0005
Epoch 11/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 11s 349ms/step - loss: 1.0005 - val_loss: 1.0005
Epoch 12/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 358ms/step - loss: 1.0005 - val_loss: 1.0005
Epoch 13/60
32/32 ━━━━━━━━━━━━━━━━━━━━ 21s 359ms/step - loss: 1.0005 - val_l

In [ ]:
best_model = search.best_estimator_

In [ ]:
def make_plot_roc_curves(qcd, bsm):
    true_val = np.concatenate((np.ones(bsm.shape[0]), np.zeros(qcd.shape[0])))
    pred_val = np.concatenate((bsm, qcd))
    fpr_loss, tpr_loss, threshold_loss = roc_curve(true_val, pred_val)
    auc_loss = auc(fpr_loss, tpr_loss)
    qcd[::-1].sort()
    return auc_loss

def eval(model,x_test,bbh_pretest,sglf_pretest):
    background_test = model.predict(x_test)
    bbh_test = model.predict(bbh_pretest)
    sglf_test = model.predict(sglf_pretest)
    mse_background = np.mean((background_test - x_test)**2, axis=(1,2))
    mse_bbh = np.mean((bbh_test - bbh_pretest)**2, axis=(1,2))
    mse_sg = np.mean((sglf_test - sglf_pretest)**2, axis=(1,2))
    print('evaluating bg vs bbh')
    bbhauc = make_plot_roc_curves(mse_background, mse_bbh)
    print('evaluation bg vs sg')
    sgauc = make_plot_roc_curves(mse_background, mse_sg)
    return bbhauc,sgauc


In [ ]:
bbhauc,sgauc=eval(best_model,x_test, bbh_test,sglf_test)
print(bbhauc,sgauc)

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
evaluating bg vs bbh
evaluation bg vs sg
0.62908858 0.42784300499999994


Testing

In [ ]:
# Load test data and normalize them
data = np.load('ligo_bb_final.npz')
test_data = data['data']
stds = np.std(test_data, axis=-1)[:, :, np.newaxis]
test_data = test_data/stds
test_data = np.swapaxes(test_data, 1, 2)
data_label = data['ids']
indices1 = np.where(data_label == 1)[0]
indices0 = np.where(data_label == 0)[0]
background_test_data = test_data[indices0]
signal_test_data = test_data[indices1]

In [ ]:
from sklearn.metrics import roc_curve, auc
def eval_test(model,background_test_data,signal_test_data):
  bg_test = model.predict(background_test_data)
  sg_test = model.predict(signal_test_data)
  mse_bg = np.mean((bg_test - background_test_data)**2, axis=(1,2))
  mse_sg = np.mean((sg_test - signal_test_data)**2, axis=(1,2))
  pred_val = np.concatenate((mse_sg, mse_bg))
  true_val = np.concatenate((np.ones(mse_sg.shape[0]), np.zeros(mse_bg.shape[0])))
  fpr_loss, tpr_loss, threshold_loss = roc_curve(true_val, pred_val)
  AUC = auc(fpr_loss, tpr_loss)
  idx = np.argmin(np.abs(tpr_loss - 0.90))    #nearest value's index at 0.9 TPR
  desired_fpr = fpr_loss[idx]    #FPR at 0.9 TPR
  return AUC, desired_fpr, mse_bg, mse_sg



AUC_, FPR_, mse_bg, mse_sg = eval_test(best_model,background_test_data,signal_test_data)

print(AUC_, FPR_,mse_bg,mse_sg)